In [1]:
# Define the file path and the Fermi energy value
file_path = 'cu_111_slab_22l_paw_band.dat'
fermi_energy = 10.448  # Replace with your Fermi energy value

# Initialize a list to hold the data
kx_ky_kz_nbnd_data = []

# Open and read the contents of the file
with open(file_path, 'r') as file:
    file_contents = file.readlines()

# Extract nbnd and nks values from the header (first line)
header_line = file_contents[0]
nbnd = int(header_line.split('nbnd=')[1].split(',')[0].strip())  # Number of nbnd values
nks = int(header_line.split('nks=')[1].split('/')[0].strip())  # Number of k-points

# Initialize an index to iterate through the file contents
i = 1  # Skip the first line (header)

# Iterate through the file, handling both coordinates and nbnd values
while i < len(file_contents):
    # Extract the kx, ky, kz coordinates (first 3 values on the current line)
    current_line = list(map(float, file_contents[i].strip().split()))
    
    # Extract the kx, ky, kz
    kx_ky_kz = current_line[:3]
    
    # Initialize an empty list to store the nbnd values
    nbnd_values = current_line[3:]  # If the current line has additional nbnd values
    
    # Continue reading lines to gather all nbnd values if they span multiple lines
    while len(nbnd_values) < nbnd:
        i += 1
        # Add more nbnd values from subsequent lines
        nbnd_values.extend(list(map(float, file_contents[i].strip().split())))
    
    # Ensure we only keep the required number of nbnd values
    nbnd_values = nbnd_values[:nbnd]
    
    # Subtract the Fermi energy from each nbnd value and round to 3 decimal places
    nbnd_values = [round(value - fermi_energy, 3) for value in nbnd_values]
    
    # Append the kx, ky, kz coordinates followed by the adjusted nbnd values to the final data list
    kx_ky_kz_nbnd_data.append(kx_ky_kz + nbnd_values)
    
    # Move to the next line that starts a new kx, ky, kz coordinate set
    i += 1

# Save the data to a CSV file
import csv
output_csv = 'band_data_fermi_subtracted_with_three_digit_prec.csv'
with open(output_csv, 'w', newline='') as csvfile:
    # Create a dynamic header based on the number of nbnd values
    header = ['kx', 'ky', 'kz'] + [f'nbnd_{i+1}' for i in range(nbnd)]
    
    writer = csv.writer(csvfile)
    writer.writerow(header)  # Write header
    writer.writerows(kx_ky_kz_nbnd_data)  # Write data rows

print(f"Data saved to {output_csv}")


Data saved to band_data_fermi_subtracted_with_three_digit_prec.csv
